In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 113.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import os

In [7]:
reviews = pd.read_parquet("/content/drive/MyDrive/reviews.parquet")

In [8]:
grouped = reviews.groupby("listing_id")["cleaned_reviews"].apply(list)

In [9]:
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
listing_embeddings = {}
for lid, texts in grouped.items():
    vectors = model.encode(texts, convert_to_numpy=True)
    listing_embeddings[lid] = np.mean(vectors, axis=0)

In [11]:
embed_df = pd.DataFrame.from_dict(listing_embeddings, orient="index").reset_index()
embed_df = embed_df.rename(columns={"index": "listing_id"})

In [12]:
os.makedirs("/content/drive/MyDrive/listing_embedding_store", exist_ok=True)

In [13]:
embed_df.to_parquet("/content/drive/MyDrive/listing_embedding_store/listing_embeddings.parquet", index=False)

In [14]:
matrix = embed_df.drop(columns=["listing_id"]).values.astype("float32")
listing_ids = embed_df["listing_id"].values

In [15]:
index = faiss.IndexFlatL2(matrix.shape[1])
index.add(matrix)

In [16]:
faiss.write_index(index, "/content/drive/MyDrive/listing_embedding_store/listing_embeddings.index")
np.save("/content/drive/MyDrive/listing_embedding_store/listing_ids.npy", listing_ids)